# Importación de bibliotecas

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Filtración de registros con Dask

In [ ]:
import dask.dataframe as dd
# --- Paso 1: Obtener los VINs usados ---
dtype_fix = {
    'bed': 'object',
    'bed_height': 'object',
    'bed_length': 'object',
    'cabin': 'object',
    'dealer_zip': 'object',
    'franchise_make': 'object',
    'sp_id': 'float64'
}
# --- 2. Obtener los VINs a excluir (Usando Pandas, es rápido) ---
print("Cargando VINs de la muestra de 150k...")
df_muestra = pd.read_csv('/content/drive/MyDrive/muestra_estratificada.csv', usecols=['vin'], dtype={'vin': 'object'})
vin_usados = set(df_muestra['vin'])
print(f"{len(vin_usados)} VINs para excluir.")

# --- 3. Cargar el dataset grande ---
print("Cargando dataset de 10GB con Dask...")
df_grande = dd.read_csv('/content/drive/MyDrive/used_cars_data.csv', dtype=dtype_fix)
df_grande_filtrado = df_grande[df_grande['price'] <= 100000]

# --- 4. Filtrar el dataset grande  ---
print("Filtrando VINs usados...")
# Aquellos registros de la muestra que no estén en el dataset original
df_no_vistos = df_grande_filtrado[~df_grande_filtrado['vin'].isin(vin_usados)]

# --- 5. Tomar la muestra (Modo LAZY) ---
# Fracción para 50k ≈ 0.0175
df_holdout_lazy = df_no_vistos.sample(frac=0.0175, random_state=123)
df_holdout_sucio = df_holdout_lazy.compute()

print(f"Muestra 'sucia' creada con {len(df_holdout_sucio)} filas.")

# --- 6. Guardar la muestra final ---
ruta_final_holdout = '/content/drive/MyDrive/holdout_SUCIO_final.csv'
df_holdout_sucio.to_csv(ruta_final_holdout, index=False)
print(f"Muestra final 'sucia' guardada en {ruta_final_holdout}")

Cargando VINs de la muestra de 150k...
149992 VINs para excluir.
Cargando dataset de 10GB con Dask...
Filtrando VINs usados...
Muestra 'sucia' creada con 49606 filas.
Muestra final 'sucia' guardada en /content/drive/MyDrive/holdout_SUCIO_final.csv


# Transformando columnas con formatos a numéricas o booleanas

In [ ]:
#Leyendo la muestra sucia
df_holdout=pd.read_csv('/content/drive/MyDrive/holdout_SUCIO_final.csv')

# back_legroom: Eliminar 'in' y dar un strip
df_holdout['back_legroom_numeric'] = pd.to_numeric(
    df_holdout['back_legroom'].str.replace(' in', '').str.strip(),errors='coerce'
).astype('float64')

print(f"\nTipo de dato de la nueva columna: {df_holdout['back_legroom_numeric'].dtype}")

# bed_length: Eliminar 'in' y dar un strip
df_holdout['bed_length_numeric'] = pd.to_numeric(
    df_holdout['bed_length'].str.replace(' in', '').str.strip(),errors='coerce'
).astype('float64')

# combine_fuel_economy: Como todos son nulos hacer el calculo combine=1/((0.55/city)+(0.45/highway))
df_holdout['combine_fuel_economy'] = 1/((0.55/df_holdout['city_fuel_economy'])+(0.45/df_holdout['highway_fuel_economy']))

def limpiar_y_convertir_booleano(valor):
    # Primero, manejar los valores nulos para que no den error
    if pd.isna(valor):
        return pd.NA  # Usamos el marcador de nulo de pandas

    # Si el valor ya es un booleano, lo convierte a 1 o 0
    if isinstance(valor, bool):
        return int(valor)

    # Si el valor es un string, lo limpiamos y lo mapeamos
    if isinstance(valor, str):
        valor_limpio = valor.lower().strip()
        if valor_limpio == 'true':
            return 1
        if valor_limpio == 'false':
            return 0

    # Si el valor es algo que no esperamos (ni nulo, ni bool, ni string "True"/"False"),
    # lo devolvemos como nulo.
    return pd.NA

# fleet:
df_holdout['fleet'] = df_holdout['fleet'].apply(limpiar_y_convertir_booleano)
df_holdout['fleet'] = df_holdout['fleet'].astype('Int64')


# frame_damaged:
df_holdout['frame_damaged'] = df_holdout['frame_damaged'].apply(limpiar_y_convertir_booleano)
df_holdout['frame_damaged'] = df_holdout['frame_damaged'].astype('Int64')

# franchise_dealer:
df_holdout['franchise_dealer']=df_holdout['franchise_dealer'].astype('int')

# front_legroom: Quitar 'in' al string y dar un strip()
df_holdout['front_legroom_numeric'] = pd.to_numeric(
    df_holdout['front_legroom'].str.replace(' in', '').str.strip(),errors='coerce'
).astype('float64')

print(f"\nTipo de dato de la nueva columna: {df_holdout['front_legroom_numeric'].dtype}")

# fuel_tank_volume: Quitar 'gal' al string y dar un strip()
df_holdout['fuel_tank_volume_numeric'] = pd.to_numeric(
    df_holdout['fuel_tank_volume'].str.replace(' gal', '').str.strip(),errors='coerce'
).astype('float64')

# has_accidents:
df_holdout['has_accidents'] = df_holdout['has_accidents'].apply(limpiar_y_convertir_booleano)
df_holdout['has_accidents'] = df_holdout['has_accidents'].astype('Int64')

# height: Eliminar el 'in' al string y dar un strip()
df_holdout['height_numeric'] = pd.to_numeric(
    df_holdout['height'].str.replace(' in', '').str.strip(),errors='coerce'
).astype('float64')

# is_cab:
df_holdout['isCab'] = df_holdout['isCab'].apply(limpiar_y_convertir_booleano)
df_holdout['isCab'] = df_holdout['isCab'].astype('Int64')

# is_cpo:
df_holdout['is_cpo']=df_holdout['is_cpo'].astype('Int64')

# is_new: Como es bool transformar a int
df_holdout['is_new']=df_holdout['is_new'].astype('int')

#is_oemcpo: Como es bool transformar a int
df_holdout['is_oemcpo']=df_holdout['is_oemcpo'].astype('Int64')

# length: Eliminar el 'in' en el string y dar un strip()
df_holdout['length_numeric'] = pd.to_numeric(
    df_holdout['length'].str.replace(' in', '').str.strip(),errors='coerce'
).astype('float64')

print(f"\nTipo de dato de la nueva columna: {df_holdout['length_numeric'].dtype}")

# maximum_seating: Eliminar el 'seats' y darle un strip() al string
df_holdout['maximum_seating_numeric'] = pd.to_numeric(
    df_holdout['maximum_seating'].str.replace(' seats', '').str.strip(),errors='coerce'
).astype('Int64')

print(f"\nTipo de dato de la nueva columna: {df_holdout['maximum_seating_numeric'].dtype}")

# owner_count:
df_holdout['owner_count'] = df_holdout['owner_count'].astype('Int64')
# Verificamos el nuevo tipo de dato
print(f"Nuevo tipo de dato: {df_holdout['owner_count'].dtype}")

#power:
# HP y RPM separados por 'hp @', puede incluso que el RPM no sea tan importante y solo una sola columna
import re
# 1. Define una función para extraer el primer número de una cadena
def extraer_valor_numerico(texto):
    # Revisa si el dato es un string
    if not isinstance(texto, str):
        return None  #Si no es cadena devolver None
    try:
        # Divide la cadena por '@' y toma la primera parte (arreglo)
        parte_principal = texto.split('@')[0]
        # Busca cualquier número (incluyendo decimales) en esa parte
        #                      signo opcional, digitos del 0 al 9, '.',digitos despues del punto o solo digitos (minimo 1)
        numeros = re.findall(r"[-+]?\d*\.\d+|\d+", parte_principal)
        # Devuelve el primer número encontrado como float
        return float(numeros[0])
    except (IndexError, ValueError):
        # Si algo falla (ej. la cadena está vacía o no tiene números), devuelve nulo
        return None
print("Creando columna numérica para 'horsepower'")
df_holdout['horsepower'] = df_holdout['power'].apply(extraer_valor_numerico).astype('float64')

# salvage:
df_holdout['salvage'] = df_holdout['salvage'].apply(limpiar_y_convertir_booleano)
df_holdout['salvage'] = df_holdout['salvage'].astype('Int64')

# theft_title:
df_holdout['theft_title'] = df_holdout['theft_title'].apply(limpiar_y_convertir_booleano)
df_holdout['theft_title'] = df_holdout['theft_title'].astype('Int64')

# torque:
# Torque y RPM separados por 'lb-ft @', puede incluso que el RPM no sea tan importante y solo una sola columna
#    Usar .apply()
df_holdout['torque_numeric'] = df_holdout['torque'].apply(extraer_valor_numerico).astype('float64')

# wheelbase: Eliminar 'in' en el string y dar strip()
df_holdout['wheelbase_numeric'] = pd.to_numeric(
    df_holdout['wheelbase'].str.replace(' in', '').str.strip(),errors='coerce'
).astype('float64')
print(f"Tipo de la nueva columna {df_holdout['wheelbase_numeric'].dtype}")

# width: Eliminar 'in' en el string y dar strip()
df_holdout['width_numeric'] = pd.to_numeric(
    df_holdout['width'].str.replace(' in', '').str.strip(),errors='coerce'
).astype('float64')
print(f"Tipo de la nueva columna {df_holdout['width_numeric'].dtype}")

#year:
from datetime import date
df_holdout['age']=date.today().year-df_holdout['year']

/tmp/ipython-input-58003352.py:2: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_holdout=pd.read_csv('/content/drive/MyDrive/holdout_SUCIO_final.csv')



Tipo de dato de la nueva columna: float64

Tipo de dato de la nueva columna: float64

Tipo de dato de la nueva columna: float64

Tipo de dato de la nueva columna: Int64
Nuevo tipo de dato: Int64
Creando columna numérica para 'horsepower'
Tipo de la nueva columna float64
Tipo de la nueva columna float64


# Transformacion Columnas Categoricas

In [ ]:
del(df_holdout['vin'])
del(df_holdout['back_legroom'])
del(df_holdout['bed_height'])
del(df_holdout['bed_length'])
del(df_holdout['cabin'])
del(df_holdout['city'])
del(df_holdout['daysonmarket'])
del(df_holdout['dealer_zip'])
del(df_holdout['description'])
del(df_holdout['exterior_color'])
del(df_holdout['front_legroom'])
del(df_holdout['fuel_tank_volume'])
del(df_holdout['height'])
del(df_holdout['interior_color'])
del(df_holdout['is_certified'])
del(df_holdout['latitude'])
del(df_holdout['length'])
del(df_holdout['listed_date'])
del(df_holdout['listing_id'])
del(df_holdout['longitude'])
del(df_holdout['main_picture_url'])
del(df_holdout['maximum_seating'])
del(df_holdout['power'])
del(df_holdout['sp_id'])
del(df_holdout['torque'])
del(df_holdout['sp_name'])
del(df_holdout['trimId'])
del(df_holdout['vehicle_damage_category'])
del(df_holdout['wheel_system_display'])
del(df_holdout['wheelbase'])
del(df_holdout['width'])
del(df_holdout['year'])

print(len(df_holdout.columns))

45


In [ ]:
#Conseguir si es pickup
df_holdout['is_pickup'] = (df_holdout['body_type'] == 'Pickup Truck').astype(int)

df_holdout['bed'].fillna('No_Aplica', inplace=True) #Llenar los nulos
df_holdout=pd.get_dummies(df_holdout, columns=['bed'], dtype=int) #Aplicar dummies a

df_holdout = pd.get_dummies(df_holdout, columns=['body_type'], dtype=int, drop_first=True)

del(df_holdout['is_pickup'])

del(df_holdout["engine_cylinders"]) #Lo mismo que engine_type

# Crear Banderas de Combustible a partir de 'fuel_type'
df_holdout['is_diesel'] = df_holdout['fuel_type'].str.contains('Diesel|Biodiesel', case=False, na=False).astype(int)
df_holdout['is_hybrid'] = df_holdout['fuel_type'].str.contains('Hybrid', case=False, na=False).astype(int)
df_holdout['is_flex_fuel'] = df_holdout['fuel_type'].str.contains('Flex Fuel', case=False, na=False).astype(int)
df_holdout['is_electric'] = df_holdout['fuel_type'].str.contains('Electric', case=False, na=False).astype(int)

# 'engine_type' SOLAMENTE para la configuración del motor
# Con RE expresar aquellos que empiecen con letras H,I,F,R,V,W y que tengan 1 o 2 dígitos adelante
df_holdout['engine_config'] = df_holdout['engine_type'].str.extract(r'([HIFRVW]\d{1,2})').fillna('Otro')

engine_counts = df_holdout['engine_config'].value_counts()
rare_engines = engine_counts[engine_counts < 100].index
#Reemplazar aquellas categorias con menos de 100 registros a 'Otro'
df_holdout['engine_config'] = df_holdout['engine_config'].replace(rare_engines, 'Otro')

del(df_holdout["fuel_type"])
del(df_holdout["engine_type"])

df_holdout = pd.get_dummies(df_holdout, columns=['engine_config'], dtype=int, drop_first=True)

del(df_holdout['franchise_make'])

del(df_holdout["listing_color"])

del(df_holdout["major_options"])

umbral = 1000

#Contar la frecuencia de cada categoría
counts = df_holdout['make_name'].value_counts()

#Categorías que están por debajo del umbral
rare_categories = counts[counts < umbral].index

df_holdout['make_name'] = df_holdout['make_name'].replace(rare_categories, 'Otro')

print(df_holdout['make_name'].value_counts())

df_holdout = pd.get_dummies(df_holdout, columns=['make_name'], dtype=int, drop_first=True)

umbral = 100
#Contar la frecuencia de cada categoría
counts = df_holdout['model_name'].value_counts()

#Categorías que están por debajo del umbral
rare_categories = counts[counts < umbral].index

df_holdout['model_name'] = df_holdout['model_name'].replace(rare_categories, 'Otro')

print(df_holdout['model_name'].value_counts())

df_holdout = pd.get_dummies(df_holdout, columns=['model_name'], dtype=int, drop_first=True)

df_holdout.drop(columns=['transmission'], inplace=True, errors='ignore')

# Extraer el TIPO principal
def get_transmission_type(text):
    text = str(text).lower()
    if 'manual' in text:
        return 'Manual'
    elif 'cvt' in text or 'variable' in text:
        return 'CVT'
    elif 'dual clutch' in text:
        return 'Dual Clutch'
    elif 'automatic' in text:
        return 'Automatic'
    else:
        return 'Otro'

df_holdout['transmission_type'] = df_holdout['transmission_display'].apply(get_transmission_type)

# Extraer las VELOCIDADES (como columna numérica)
df_holdout['transmission_speeds'] = df_holdout['transmission_display'].str.extract(r'(\d{1,2})')
df_holdout['transmission_speeds'] = pd.to_numeric(df_holdout['transmission_speeds'], errors='coerce')

# Rellenar los nulos (ej. 'Automatic' a secas) con la mediana de las velocidades
df_holdout['transmission_speeds'].fillna(df_holdout['transmission_speeds'].median(), inplace=True)

# Extraer la bandera 'is_overdrive' (booleano)
df_holdout['is_overdrive'] = df_holdout['transmission_display'].str.contains('Overdrive', na=False).astype(int)

# Eliminar la columna 'transmission_display' original
df_holdout.drop(columns=['transmission_display'], inplace=True)

#Conseguir dummies booleanos para el tipo de transmision
df_holdout = pd.get_dummies(df_holdout, columns=['transmission_type'], dtype=int, drop_first=True)

del(df_holdout['trim_name']) #demasiadas categorias, eliminar

df_holdout=pd.get_dummies(df_holdout, columns=['wheel_system'],dtype=int, drop_first=True)


/tmp/ipython-input-1039618913.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_holdout['bed'].fillna('No_Aplica', inplace=True) #Llenar los nulos


make_name
Ford             7909
Otro             6925
Chevrolet        6189
Toyota           4045
Nissan           3705
Honda            3448
Jeep             2833
Hyundai          2268
Kia              1820
RAM              1685
GMC              1683
Dodge            1533
Volkswagen       1283
Subaru           1121
Buick            1084
Mercedes-Benz    1047
BMW              1028
Name: count, dtype: int64
model_name
Otro              10736
F-150              2177
Silverado 1500     1227
1500               1139
Equinox            1030
                  ...  
Camry Hybrid        106
K5                  104
Transit Cargo       102
Q50                 101
MKZ                 100
Name: count, Length: 118, dtype: int64


/tmp/ipython-input-1039618913.py:89: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_holdout['transmission_speeds'].fillna(df_holdout['transmission_speeds'].median(), inplace=True)


# Manejo de nulos

In [ ]:
muestra_limpia=pd.read_csv('/content/drive/MyDrive/ddf4_limpio.csv')

# Crear el diccionario de imputación a partir de el
values = {}

# Relleno con Cero (Booleanas Dispersas)
bool_sparse_cols = ['is_cpo', 'is_oemcpo', 'isCab']
for col in bool_sparse_cols:
    values[col] = 0

# Relleno con Moda
bool_dense_cols = ['fleet', 'frame_damaged', 'has_accidents', 'salvage', 'theft_title']
for col in bool_dense_cols:
    if col in muestra_limpia.columns:
        # Obtenemos la moda del DataFrame limpio
        values[col] = muestra_limpia[col].mode()[0]

# --- Relleno con Mediana (Numéricas) ---
numeric_cols_median = [
    'city_fuel_economy', 'highway_fuel_economy', 'engine_displacement',
    'horsepower', 'mileage', 'seller_rating', 'back_legroom_numeric',
    'front_legroom_numeric', 'fuel_tank_volume_numeric', 'height_numeric',
    'length_numeric', 'torque_numeric', 'wheelbase_numeric', 'width_numeric',
    'owner_count', 'maximum_seating_numeric'
]


for col in numeric_cols_median:
    if col in muestra_limpia.columns:
        # Obtenemos la mediana del DataFrame limpio
        values[col] = muestra_limpia[col].median()

values['bed_length_numeric'] = 0

# Rellenar los nulos del hold-out usando valores imputados
print("Rellenando nulos del hold-out...")
df_holdout_limpio = df_holdout.fillna(value=values)

# Hacer el calculo sobre los que hacian falta
df_holdout_limpio['combine_fuel_economy'] = 1 / ((0.55 / df_holdout_limpio['city_fuel_economy']) + (0.45 / df_holdout_limpio['highway_fuel_economy']))

# --- Verificación Final ---
total_nulos_final = df_holdout_limpio.isnull().sum().sum()
if total_nulos_final == 0:
    print(f"\nDataFrame está 100% limpio (0 nulos).")
else:
    print(f"\nAtención: Aún quedan {total_nulos_final} nulos por revisar.")


Rellenando nulos del hold-out...

DataFrame está 100% limpio (0 nulos).


In [ ]:
# --- Lista de columnas del modelo ---
model_columns = ['mileage', 'fuel_tank_volume_numeric', 'torque_numeric', 'age', 'horsepower',
 'wheel_system_AWD', 'back_legroom_numeric', 'height_numeric',
 'width_numeric', 'engine_displacement', 'wheel_system_FWD',
 'body_type_SUV / Crossover', 'maximum_seating_numeric', 'is_new',
 'make_name_BMW', 'length_numeric', 'model_name_Otro', 'is_diesel',
 'body_type_Pickup Truck', 'wheelbase_numeric', 'make_name_Dodge',
 'front_legroom_numeric', 'make_name_Cadillac', 'combine_fuel_economy',
 'savings_amount', 'make_name_Ford', 'wheel_system_4X2', 'make_name_Otro',
 'seller_rating', 'make_name_Kia', 'engine_config_V8', 'highway_fuel_economy',
 'is_electric', 'make_name_Volvo', 'engine_config_I4', 'engine_config_I6',
 'city_fuel_economy', 'model_name_Durango', 'make_name_Mercedes-Benz',
 'make_name_Lexus']


X_holdout = df_holdout_limpio.drop(columns="price", errors='ignore')
y_holdout = df_holdout_limpio["price"]

# --- Alinear X_holdout con las columnas del modelo ---
# .reindex() hace 3 cosas:
#  1. Mantiene las columnas que coinciden.
#  2. Añade las columnas faltantes
#  3. Rellena esas columnas nuevas con 0 (lo cual es correcto).
#  4. Elimina columnas extra que no estaban en el entrenamiento (si las hubiera).
X_holdout_alineado = X_holdout.reindex(columns=model_columns, fill_value=0)

In [ ]:
# Guardar el resultado final
df_holdout_final = pd.concat([X_holdout_alineado, y_holdout], axis=1)

ruta_holdout_limpio_final = '/content/drive/MyDrive/holdout_limpio_final.csv'
df_holdout_final.to_csv(ruta_holdout_limpio_final, index=False)

print(f"Set de validación final guardado.")


Set de validación final guardado.
